In [1]:
import cv2
import numpy as np
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input,decode_predictions
from keras.models import load_model


#loading pretrained inception v3 model
base_model=InceptionV3(weights='imagenet',include_top=False)
#adding custom classifiction layers on top of inception v3
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
# x=Dense(512,activation='relu')(x)
# x=Dense(128,activation='relu')(x)
# predictions=Dense(10,activation='softmax')(x)
predictions=Dense(2,activation='softmax')(x)
#combinig base model with custom layers
model=Model(inputs=base_model.input,outputs=predictions)
#freezing base model layers
for layer in base_model.layers:
    layer.trainable=False

#load the distracted model
load_model('C:/Users/master/Documents/mega_project/distraction_inception.h5')

#start webcam capture
cap=cv2.VideoCapture(0)

#defining labesls for distraction classes
# labels={
#    0:'Safe driving',
#     1:'Texting - right',
#     2:'Talking on the phone - right',
#     3:'Texting - left',
#     4:'Talking on the phone - left',
#     5:'Operating the radio',
#     6:'Drinking',
#     7:'Reaching behind',
#     8:'Hair and makeup',
#     9:'Talking to passenger',
# }

labels={
    0:'Safe driving',
    1:'distracted',
}
while True:
    #capturing frame by frame
    ret,frame=cap.read()
    if not ret:
        break

    #preprocess frame for inception v3
    frame_resized=cv2.resize(frame,(299,299))
    frame_resized=cv2.cvtColor(frame_resized,cv2.COLOR_BGR2RGB)
    frame_resized=np.expand_dims(frame_resized,axis=0)
    frame_resized=preprocess_input(frame_resized)

    #predict distraction using inception v3 model
    prediction=model.predict(frame_resized)
    label=labels[np.argmax(prediction)]

    #display label on the frame
    cv2.putText(frame,"distraction:{}".format(label),(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

    #display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1)  & 0xFF == ord('q'):
        break

#release the capture
cap.release()
cv2.destroyAllWindows()




1/1 [==============================] - 0s 280ms/step
